In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langchain_teddynote.tools.tavily import TavilySearch

In [3]:
from langchain.schema import HumanMessage, AIMessage
from langchain_core.messages import ToolMessage

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [5]:
llm = ChatOpenAI( 
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

NameError: name 'key' is not defined

In [ ]:
tool = TavilySearch(max_results=1)      # 검색 도구 생성
tools = [tool]                          # 도구 목록에 넣기  

In [ ]:
# LLM 에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    print(f'=='*50)
    print('===== chatbot() 함수 시작 =====')
    print(f"chatbot() 으로 넘어온 메시지 :")
    print(state['messages'])
    print()

    answer = llm_with_tools.invoke(state['messages'])

    print(f'[도구 사용 LLM 실행 결과 content]: {answer.content}')
    print(f'[도구 사용 LLM 실행 결과 answer]: {answer}')
    print(f'[도구 사용 LLM 실행 결과 additional_kwargs]: {answer.additional_kwargs}')

    print('===== chatbot() 함수  끝 =====')
    print(f'=='*50)
    print()

    return {'messages': [answer]}

In [ ]:
question = "대구의 유명한 맛집을 알려줘."

state1 = State(messages=[('user', question)])
answer_state = chatbot(state1)

===== chatbot() 함수 시작 =====
chatbot() 으로 넘어온 메시지 :
[('user', '대구의 유명한 맛집을 알려줘.')]

[도구 사용 LLM 실행 결과 content]: 
[도구 사용 LLM 실행 결과 answer]: content='' additional_kwargs={'tool_calls': [{'id': 'call_F6dcOhU7VEXMJrY8EXy4027i', 'function': {'arguments': '{"query":"대구 유명 맛집"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 102, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f7fed7c7-678c-40e7-852d-5ea7ad7652e8-0' tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '대구 유명 맛집'}, 'id': 'call_F6dcOhU7VEXMJrY8EXy4027i', 'type': 'tool_call'}] usage_metadata={'input_tokens': 102, 'ou

In [ ]:
answer_state

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_F6dcOhU7VEXMJrY8EXy4027i', 'function': {'arguments': '{"query":"대구 유명 맛집"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 102, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f7fed7c7-678c-40e7-852d-5ea7ad7652e8-0', tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '대구 유명 맛집'}, 'id': 'call_F6dcOhU7VEXMJrY8EXy4027i', 'type': 'tool_call'}], usage_metadata={'input_tokens': 102, 'output_tokens': 22, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [ ]:
from langgraph.graph import StateGraph, START, END
import json

def route_tools(state: State):
    if messages := state.get('messages', []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    print()
    print('===== 라우터 노드 =====')
    print(f"라우터 노드로 넘어온 chatbot() 에서 생성한 state 메시지: ")
    print(ai_message.additional_kwargs)

    if hasattr(ai_message, 'tool_calls') and len(ai_message.tool_calls) > 0:
        print(f'라우터 [tools] -> {ai_message.tool_calls}')
        print(f'라우터 리턴: tools')
        print('===== 라우터 노드 끝 =====')
        print()

        return 'tools'
    
    
    print(f'라우터 리턴: {END}')
    print('===== 라우터 노드 끝 =====')
    print()
    
    return END    

In [ ]:
state_with_tools = {
    "messages": [
        HumanMessage(
            content="대한민국 수도에 대해서 검색해줘",
            id='6e0ff6a0-d790-42a0-ae24-28172c7feabc'
        ),
        AIMessage(
            content="",
            tool_calls=[{
                "name": "tavily_web_search",                        
                "args": {"query": "대한민국 수도"},
                "id": "call_k6ljrhTRP8dz8FwNrtbyyG3K"
            }],
            additional_kwargs={
                "tool_calls": [{
                    "id": "call_k6ljrhTRP8dz8FwNrtbyyG3K",
                    "function": {
                        "arguments": json.dumps({"query": "대한민국 수도"}),
                        "name": "tavily_web_search"                     
                    },
                    "type": "function"
                }],
                "refusal": None
            },
            response_metadata={
                "token_usage": {
                    "completion_tokens": 20,
                    "prompt_tokens": 99,
                    "total_tokens": 119
                },
                "model_name": "gpt-4o-mini-2024-07-18"
            },
            id='run-164b42b2-a16c-4ea8-8a2e-94da55991426-0'
        )
    ]
}


route_tools(state_with_tools)


===== 라우터 노드 =====
라우터 노드로 넘어온 chatbot() 에서 생성한 state 메시지: 
{'tool_calls': [{'id': 'call_k6ljrhTRP8dz8FwNrtbyyG3K', 'function': {'arguments': '{"query": "\\ub300\\ud55c\\ubbfc\\uad6d \\uc218\\ub3c4"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}
라우터 [tools] -> [{'name': 'tavily_web_search', 'args': {'query': '대한민국 수도'}, 'id': 'call_k6ljrhTRP8dz8FwNrtbyyG3K', 'type': 'tool_call'}]
라우터 리턴: tools
===== 라우터 노드 끝 =====



'tools'

In [ ]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """Run tools requested in the last AIMessage node"""

    def __init__(self, tools: list) -> None:
        self.tools_list = {tool.name: tool for tool in tools}
        print(f'======================================================')
        print('[BasicToolNode]')
        print('도구 호출 생성자')
        print(f'tools_list: {self.tools_list}')
        print(f'======================================================')
    
    def __call__(self, inputs: dict):
        if messages := inputs.get('messages', []):
            message = messages[-1]
        else:
            raise ValueError('No message found in input')
        
        print()
        print(f'======================================================')
        print('[BasicToolNode] call')
        #print('도구 호출로 갔을 때')
        #print('message.tool_calls:', message.tool_calls)
        #print(f'======================================================')

        outputs = []

        for tool_call in message.tool_calls:    # message의 tool_calls 속성이 있는것은 도구 호출을 필요한 메시지가 있는 경우
            print(f'도구 호출이 필요한 경우: ')
            # print(f'도구 호출 : {tool_call}')
            print(f"도구 호출 이름: {tool_call['name']}")
            print(f"도구 호출 인자 : {tool_call['args']}")

            tool_result = self.tools_list[tool_call['name']].invoke(tool_call['args'])
            print(f'도구 호출 결과 : {tool_result}')                                            
        
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result, ensure_ascii=False), 
                    name=tool_call['name'], 
                    tool_call_id=tool_call['id']
                )
            )
            
        print('[BasicToolNode] call 끝')
        print(f'======================================================')
        print()

        return {'messages': outputs}    

In [ ]:
# 도구 노드 생성
tool_node = BasicToolNode(tools=[tool])   

[BasicToolNode]
도구 호출 생성자
tools_list: {'tavily_web_search': TavilySearch(client=<tavily.tavily.TavilyClient object at 0x00000243A22A5010>, max_results=1)}


In [ ]:
from langchain_core.runnables import RunnableLambda

tool_node_runnable = RunnableLambda(tool_node)

In [ ]:
sample_input = State(
    messages=[
        HumanMessage(content='대구 맛집을 알려줘'),
        AIMessage(
            content='',
            tool_calls=[{
                    'name': 'tavily_web_search', 
                    'args': {'query': '대구 맛집'},
                    'id': '99c32be8-e500-4b2c-9675-34ab45fa6ab5'
                }]
        )
    ]
)

result = tool_node(sample_input)

for msg in result["messages"]:
        print(f"- {msg.name}: {msg.content}")


[BasicToolNode] call
도구 호출이 필요한 경우: 
도구 호출 이름: tavily_web_search
도구 호출 인자 : {'query': '대구 맛집'}
도구 호출 결과 : [{'title': '대구 맛집 베스트10 추천 2025 가성비 현지인 숨은 순위', 'url': 'https://jdblue2022.tistory.com/entry/대구-맛집-베스트10', 'content': '바다문의 세상소식 맛집/경상맛집 오늘은 팔공산과 이월드 83 타워, 서문시장으로 유명한 대구광역시의 소문난 대구 맛집 베스트 10곳을 소개해드리고자 하는데\xa0양대 포털의\xa0플레이스를 중심으로 순위를 취합하여 가성비 좋고 현지인들이 많이 찾는 대구 숨은 맛집과 대표적인 대구 먹거리인 대구 막창과 연탄 불고기, 중화 비빔밥 등을\xa0묶어서 대구 맛집 순위를 정리해드리고자 합니다. 기본적인 맛집 선정 기준은 양대 검색 포털인 네이버(50%)를 비롯해 구글(50%)의 플레이스 순위를 반영하여 선정하였습니다. 대구 맛집 베스트10 주요 메뉴 네이버 50%+구글 50% 반영 왕거미식당(네이버 2위, 구글 3위) 매장 위치 매장 위치 매장 위치 매장 위치 주말이나 공휴일의 경우 웨이팅이 많이 걸리기 때문에 줄 서서 먹는 중식당으로 탕수육과 야끼우동에 대한 평이 좋아 중식 요리 좋아하는 분들이 가볼 만한 대구 현지인 가성비 맛집입니다. 매장 위치 매장 위치 매장 위치 매장 위치 매장 위치 매장 위치 세상소식', 'score': 0.8467682, 'raw_content': "Published Time: 2025-01-26T07:00:57+09:00\n본문 바로가기\n바다문의 세상소식\n검색\n홈\n태그\n맛집/경상맛집\n대구 맛집 베스트10 추천 2025 가성비 현지인 숨은 순위\nby 바다문 2025. 1. 26.\n오늘은 팔공산과 이월드 83 타워, 서문시장으로 유명한 대구광역시의 소문난 대구 맛집 베스트 10곳을 소개해드리고자 하는데\xa0양대 포털의\xa0플레이